In [42]:
import pandas as pd
import numpy as np
from datetime import datetime

Exploratory Data Analysis

In [43]:
df = pd.read_csv("data/Airplane_Crashes_and_Fatalities_Since_1908.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5268 entries, 0 to 5267
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         5268 non-null   int64  
 1   Date          5268 non-null   object 
 2   Time          3049 non-null   object 
 3   Location      5248 non-null   object 
 4   Operator      5250 non-null   object 
 5   Flight #      1069 non-null   object 
 6   Route         3561 non-null   object 
 7   Type          5241 non-null   object 
 8   Registration  4933 non-null   object 
 9   cn/In         4040 non-null   object 
 10  Aboard        5246 non-null   float64
 11  Fatalities    5256 non-null   float64
 12  Ground        5246 non-null   float64
 13  Summary       4878 non-null   object 
dtypes: float64(3), int64(1), object(10)
memory usage: 576.3+ KB


,index,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary
0,0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,1,07/12/1912,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
2,2,08/06/1913,NaN,"Victoria, British Columbia, Canada",Private,-,NaN,Curtiss seaplane,NaN,NaN,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...
3,3,09/09/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,14.0,0.0,The airship flew into a thunderstorm and encou...
4,4,10/17/1913,10:30,"Near Johannisthal, Germany",Military - German Navy,NaN,NaN,Zeppelin L-2 (airship),NaN,NaN,30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...


In [45]:
# Número de filas y columnas
print("Dimensiones:", df.shape)

Dimensiones: (5268, 14)


In [46]:
# Estadísticas descriptivas
print(df.describe())

            index       Aboard   Fatalities       Ground
count  5268.00000  5246.000000  5256.000000  5246.000000
mean   2633.50000    27.554518    20.068303     1.608845
std    1520.88494    43.076711    33.199952    53.987827
min       0.00000     0.000000     0.000000     0.000000
25%    1316.75000     5.000000     3.000000     0.000000
50%    2633.50000    13.000000     9.000000     0.000000
75%    3950.25000    30.000000    23.000000     0.000000
max    5267.00000   644.000000   583.000000  2750.000000


In [47]:
# Conteo de valores por columna
print(df.nunique())

index           5268
Date            4753
Time            1005
Location        4303
Operator        2476
Flight #         724
Route           3243
Type            2446
Registration    4905
cn/In           3707
Aboard           239
Fatalities       191
Ground            50
Summary         4673
dtype: int64


In [48]:
# Estadísticas también para categóricas
print(df.describe(include='object'))

              Date   Time           Location  Operator Flight #     Route  \
count         5268   3049               5248      5250     1069      3561   
unique        4753   1005               4303      2476      724      3243   
top     09/11/2001  15:00  Sao Paulo, Brazil  Aeroflot        -  Training   
freq             4     32                 15       179       67        81   

                Type Registration cn/In                  Summary  
count           5241         4933  4040                     4878  
unique          2446         4905  3707                     4673  
top     Douglas DC-3           49   178  Crashed during takeoff.  
freq             334            3     6                       15  


Normalize and renaming columns

In [49]:
# Normalizamos nombres de columnas quitando espacios extra
df.columns = [c.strip() for c in df.columns]
# index,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary
df = df.rename(columns={'index':'Index','Flight #':'FlightNumber'})

Exploratory Quality Check

a. Tipo de dato y valores faltantes

In [31]:
print("Tipo de dato por columna")
print(df.dtypes)

Tipo de dato por columna
Index             int64
Date             object
Time             object
Location         object
Operator         object
FlightNumber     object
Route            object
Type             object
Registration     object
cn/In            object
Aboard          float64
Fatalities      float64
Ground          float64
Summary          object
dtype: object


In [50]:
print("Total valores nulos por columna")
print(df.isnull().sum())

Total valores nulos por columna
Index              0
Date               0
Time            2219
Location          20
Operator          18
FlightNumber    4199
Route           1707
Type              27
Registration     335
cn/In           1228
Aboard            22
Fatalities        12
Ground            22
Summary          390
dtype: int64


In [51]:
print("Porcentaje de valores nulos por columna")
print(df.isnull().mean() * 100)

Porcentaje de valores nulos por columna
Index            0.000000
Date             0.000000
Time            42.122248
Location         0.379651
Operator         0.341686
FlightNumber    79.707669
Route           32.403189
Type             0.512528
Registration     6.359150
cn/In           23.310554
Aboard           0.417616
Fatalities       0.227790
Ground           0.417616
Summary          7.403189
dtype: float64


b. unique values by column

In [23]:
for col in df.columns:
    print(f"{col}: {df[col].nunique()} valores únicos")

Index: 5268 valores únicos
Date: 4753 valores únicos
Time: 1005 valores únicos
Location: 4303 valores únicos
Operator: 2476 valores únicos
FlightNumber: 724 valores únicos
Route: 3243 valores únicos
Type: 2446 valores únicos
Registration: 4905 valores únicos
cn/In: 3707 valores únicos
Aboard: 239 valores únicos
Fatalities: 191 valores únicos
Ground: 50 valores únicos
Summary: 4673 valores únicos


c. INVALID FORMATS

Date and Time

In [ ]:
#  'Date' to datetime, forcing error to NaT
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

#count invalid dates
invalid_dates_count = df['Date'].isna().sum()

print(f"number of invalid dates: {invalid_dates_count}")

print(f"\nValid Dates Range:")
print(f"Minimun Date: {df['Date'].min()}")
print(f"Maximiun Date: {df['Date'].max()}")

number of invalid dates: 0

Valid Dates Range:
Minimun Date: 1908-09-17 00:00:00
Maximiun Date: 2009-06-08 00:00:00


In [ ]:
# Verificar fechas menores a 1908 o mayores a 2025
fecha_minima = pd.to_datetime('1908-01-01')
fechas_menores_1908 = df[df['Date'] < fecha_minima]
print(f"Número de fechas menores a 1908: {len(fechas_menores_1908)}")
fecha_maxima = pd.to_datetime('2025-09-17')
fechas_mayores_2025 = df[df['Date'] > fecha_maxima] 
print(f"Número de fechas mayores a 2025: {len(fechas_mayores_2025)}")


Número de fechas menores a 1908: 0
Número de fechas mayores a 2025: 0


In [79]:
#Time

print(f"Valores faltantes: {df['Time'].isna().sum()} ({df['Time'].isna().mean()*100:.1f}%)")

# Verify format HH:MM with regular expressions
weird_format = df['Time'].dropna()[~df['Time'].dropna().str.match(r'^\d{2}:\d{2}$')]
print(f"Format not HH:MM: {len(weird_format)}")


Valores faltantes: 0 (0.0%)
Format not HH:MM: 2238


d. NULL VALUES

Filling null values of category columns

In [ ]:
# Columnas categóricas donde aplicar "sin especificar"
category_columns = ['Time', 'Location', 'Operator', 'FlightNumber', 'Route', 
                       'Type', 'Registration', 'cn/In', 'Summary']

# Llenar valores nulos con "sin especificar" en todas las columnas categóricas
for columna in category_columns:
    df[columna] = df[columna].fillna('not specified')

# Verificar resultados
print("null values after filling:")
print(df[category_columns].isnull().sum())

null values after filling:
Time            0
Location        0
Operator        0
FlightNumber    0
Route           0
Type            0
Registration    0
cn/In           0
Summary         0
dtype: int64


creating new columns for NaN  values of numeric columns

In [98]:

# Null values True: 1, False: 0
df['flag_missing_aboard'] = df['Aboard'].isna().astype(int)  
df['flag_missing_fatalities'] = df['Fatalities'].isna().astype(int)  
df['flag_missing_ground'] = df['Ground'].isna().astype(int)     

print(f"Null values in Aboard: {df['flag_missing_aboard'].sum()}")
print(f"Null values in Fatalities: {df['flag_missing_fatalities'].sum()}")
print(f"Null values in Ground: {df['flag_missing_ground'].sum()}")  


Null values in Aboard: 22
Null values in Fatalities: 12
Null values in Ground: 22


In [121]:
df.isnull().sum()

Index                           0
Date                            0
Time                            0
Location                        0
Operator                        0
FlightNumber                    0
Route                           0
Type                            0
Registration                    0
cn/In                           0
Aboard                         22
Fatalities                     12
Ground                         22
Summary                         0
flag_missing_aboard             0
flag_missing_fatalities         0
flag_missing_ground             0
flag_missing_aboard_int         0
flag_missing_fatalities_int     0
Location_clean                  0
Route_category                  0
Type_clean                      0
Type_standardized               0
is_dup_key                      0
dtype: int64

Category Formats

In [ ]:
category_formats = ['Location', 'Operator', 'Type', 'Route', 'Registration', 'cn/In']

# 2. Function to find variations
def find_variations(column_name, threshold=10):
    print(f"\n=== VARIATIONS IN {column_name.upper()} ===")
    count = df[column_name].value_counts()
    
    # Show values with few occurrences (potential variations)
    variations = count[count <= threshold]
    print(f"Values with less than {threshold} occurrences:")
    print(variations)
    
    return variations

# Review variations in all columns
for column in category_formats:
    find_variations(column, threshold=5)


=== VARIATIONS IN LOCATION ===
Values with less than 5 occurrences:
Location
Near Kathmandu, Nepal                5
Kansas City, Kansas                  5
Elizabeth, New Jersey                5
Seattle, Washington                  5
Fairbanks, Alaska                    5
                                    ..
Near Charana, Bolivia                1
Monte Matto, Italy                   1
Misaki Mountain, Japan               1
Angelholm, Sweden                    1
State of Arunachal Pradesh, India    1
Name: count, Length: 4264, dtype: int64

=== VARIATIONS IN OPERATOR ===
Values with less than 5 occurrences:
Operator
Military - Indonesian Air Force        5
Aeronaves de Mexico                    5
Syndicato Condor                       5
LANSA                                  5
Military - Nicaraguan Air Force        5
                                      ..
Military - Argentine Navy              1
Richland Flying Service - Air Taxii    1
Harbor Airlines - Air Taxi             1
Aerovi

In [108]:
import re

def clean_location(location):
    """
    Automated cleaning for worldwide locations
    Maintains original format but removes common inconsistencies
    """
    if pd.isna(location):
        return location
    
    location = str(location)
    
    # Remove common prefixes and unnecessary words
    remove_patterns = [
        r'^near\s+', r'^approx\s+', r'^around\s+', r'^close to\s+',
        r'\s+\(.*?\)',  # Remove anything in parentheses
        r'\s+\[.*?\]',  # Remove anything in brackets
        r'\s+off\s+',   # Remove 'off' 
        r'\s+area$', r'\s+region$',  # Remove area/region suffixes
    ]
    
    for pattern in remove_patterns:
        location = re.sub(pattern, '', location, flags=re.IGNORECASE)
    
    # Standardize country names (minimal essential mapping)
    country_mapping = {
        r'USA|U\.S\.A\.|United States|U\.S\.': 'USA',
        r'UK|U\.K\.|United Kingdom|England|Britain': 'UK',
        r'Russia|USSR|Soviet Union': 'Russia',
        r'\.': '',  # Remove dots from abbreviations
    }
    
    for pattern, replacement in country_mapping.items():
        location = re.sub(pattern, replacement, location, flags=re.IGNORECASE)
    
    # Clean up formatting
    location = location.strip()
    location = re.sub(r'\s+', ' ', location)  # Remove extra spaces
    location = location.title()  # Title case
    
    return location

# Apply automated cleaning
df['Location_clean'] = df['Location'].apply(clean_location)

# Verify results
print("Location cleaning completed!")
print("Before vs After examples:")
for i in range(5):
    print(f"{df['Location'].iloc[i]} -> {df['Location_clean'].iloc[i]}")

Location cleaning completed!
Before vs After examples:
Fort Myer, Virginia -> Fort Myer, Virginia
AtlantiCity, New Jersey -> Atlanticity, New Jersey
Victoria, British Columbia, Canada -> Victoria, British Columbia, Canada
Over the North Sea -> Over The North Sea
Near Johannisthal, Germany -> Johannisthal, Germany


In [109]:
print("=== ROUTE COLUMN ANALYSIS ===")
print("Unique values in Route (top 15):")
print(df['Route'].value_counts().head(15))

=== ROUTE COLUMN ANALYSIS ===
Unique values in Route (top 15):
Route
not specified                 1707
Training                        81
Sightseeing                     29
Test flight                     17
Test                             6
Sao Paulo - Rio de Janeiro       5
Saigon - Paris                   4
Sao Paulo - Porto Alegre         4
Bogota - Barranquilla            4
Villavicencio - Mitu             4
Huambo - Luanda                  3
Quito - Cuenca                   3
Davao - Manila                   3
Rio de Janeiro - Sao Paulo       3
Burbank - Oakland                3
Name: count, dtype: int64


In [ ]:
# delete the spaces at the beginning and end
df['Type_clean'] = df['Type'].str.strip()

In [118]:
def categorize_route(route):
    """
    Minimal route categorization - just the essentials
    """
    if pd.isna(route) or (route_str := str(route).strip()) == '':
        return 'Not Specified'
    
    # Only handle the most common cases
    if route_str.lower() == 'not specified':
        return 'Not Specified'
    elif route_str.lower() == 'test':
        return 'Test Flight'
    elif route_str.lower() == 'test flight':
        return 'Test Flight'
    elif route_str.lower() in ['training', 'sightseeing', 'demonstration']:
        return route_str.title()
    
    # For routes, just do basic cleaning
    if '-' in route_str or ' to ' in route_str.lower():
        return re.sub(r'\s+to\s+', ' - ', route_str, flags=re.IGNORECASE).title()
    
    return route_str.title()


df['Route_category'] = df['Route'].apply(categorize_route)

In [110]:
print("=== TYPE COLUMN ANALYSIS ===") 
print("Unique values in Type (top 15):")
print(df['Type'].value_counts().head(15))

=== TYPE COLUMN ANALYSIS ===
Unique values in Type (top 15):
Type
Douglas DC-3                                334
de Havilland Canada DHC-6 Twin Otter 300     81
Douglas C-47A                                74
Douglas C-47                                 62
Douglas DC-4                                 40
Yakovlev YAK-40                              37
Antonov AN-26                                36
Junkers JU-52/3m                             32
Douglas C-47B                                29
De Havilland DH-4                            28
not specified                                27
Douglas DC-6B                                27
Breguet 14                                   23
Curtiss C-46A                                21
Douglas C-47-DL                              20
Name: count, dtype: int64


In [119]:
def standardize_manufacturers(aircraft_type):
    """
    Standardize manufacturer names to consistent capitalization
    """
    if pd.isna(aircraft_type) or (type_str := str(aircraft_type).strip()) == '':
        return 'Not Specified'
    
    if type_str.lower() == 'not specified':
        return 'Not Specified'
    
    # Estandarizar nombres de fabricantes (similar al 'to' de rutas)
    manufacturer_standardization = {
        r'\bde havilland\b': 'De Havilland',
        r'\bdouglas\b': 'Douglas',
        r'\bboeing\b': 'Boeing', 
        r'\bairbus\b': 'Airbus',
        r'\bantonov\b': 'Antonov',
        r'\byakovlev\b': 'Yakovlev',
        r'\bjunkers\b': 'Junkers',
        r'\bcurtiss\b': 'Curtiss',
        r'\bbreguet\b': 'Breguet'
    }
    
    # Aplicar estandarización de fabricantes
    for pattern, replacement in manufacturer_standardization.items():
        type_str = re.sub(pattern, replacement, type_str, flags=re.IGNORECASE)
    
    return type_str

# Apply manufacturer standardization
df['Type_standardized'] = df['Type'].apply(standardize_manufacturers)

# Verify results
print("Manufacturer standardization completed!")
print("\nTop 15 standardized types:")
print(df['Type_standardized'].value_counts().head(15))

# Check specific examples
print("\nBefore vs After examples:")
examples = [
    'de Havilland Canada DHC-6 Twin Otter 300',
    'De Havilland DH-4', 
    'douglas dc-3',
    'Douglas C-47A'
]

for example in examples:
    result = standardize_manufacturers(example)
    print(f"{example} -> {result}")

Manufacturer standardization completed!

Top 15 standardized types:
Type_standardized
Douglas DC-3                                334
De Havilland Canada DHC-6 Twin Otter 300     81
Douglas C-47A                                74
Douglas C-47                                 62
Douglas DC-4                                 40
Yakovlev YAK-40                              37
Antonov AN-26                                36
Junkers JU-52/3m                             32
Douglas C-47B                                29
De Havilland DH-4                            28
Douglas DC-6B                                27
Not Specified                                27
Breguet 14                                   23
Curtiss C-46A                                21
Douglas C-47-DL                              20
Name: count, dtype: int64

Before vs After examples:
de Havilland Canada DHC-6 Twin Otter 300 -> De Havilland Canada DHC-6 Twin Otter 300
De Havilland DH-4 -> De Havilland DH-4
douglas dc-3 -> D

Duplicated Records

In [120]:
# --- Revisar duplicados exactos en todo el dataset ---
n_exact_dups = df.duplicated().sum()
print(f"Duplicados EXACTOS en todo el registro: {n_exact_dups}")

# --- Revisar duplicados por clave lógica (Date + Location + Operator + Type) ---
key_cols = ['Date', 'Location', 'Operator', 'Type']

# Filtrar filas que aparecen más de una vez con la misma clave
df['is_dup_key'] = df.duplicated(subset=key_cols, keep=False)

# Cuántos duplicados hay según esa clave
n_dup_key = df['is_dup_key'].sum()
print(f"Duplicados por clave {key_cols}: {n_dup_key}")

# Mostrar ejemplos de duplicados
dup_examples = df[df['is_dup_key']].sort_values(key_cols).head(20)
display(dup_examples)


Duplicados EXACTOS en todo el registro: 0
Duplicados por clave ['Date', 'Location', 'Operator', 'Type']: 0


,Index,Date,Time,Location,Operator,FlightNumber,Route,Type,Registration,cn/In,...,flag_missing_aboard,flag_missing_fatalities,flag_missing_ground,flag_missing_aboard_int,flag_missing_fatalities_int,Location_clean,Route_category,Type_clean,Type_standardized,is_dup_key
